<a href="https://colab.research.google.com/github/ajimper/ProAstronomia/blob/main/Copia_de_maqueta_estrellas_cercanas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Construyendo un modelo para una maqueta de las estrellas cercanas
=================================

In [ ]:
#from IPython.display import FileLink, FileLinks

## La tarea
- **Preparar una maqueta del vecindario estelar para nuestro Museo, que indique su posición en la galaxia, sus parámetros físicos elementales, así como sus sistemas planetarios, partiendo de datos reales actualizados.**
- **Fabricar el modelo y documentar los sistemas estelares que representa**

![Maqueta de las estrellas cercanas](./img/tarea.png "Maqueta de las estrellas cercanas")

Hay que representar una cantidad razonable de estrellas (que habrá que determinar, limitando la distancia de corte de nuestro subconjunto de estrellas) en un sistema de referencia cartesiano (para facilitar su construción en una escala reducida a cm), a partir de coordenadas galácticas con centro en el Sol, o sea, el plano representado será el que pasa por el Sol, paralelo al plano galáctico. Y ello es porque vamos a representar una vecindad de estrellas en la **Galaxia**, y es como si estuviéramos mirando desde fuera del Sistema Solar, por ello no nos interesa lo que se vería desde la Tierra (sistemas altaazimutal o ecuatorial con un marco de referencia geocéntrico o topocéntrico), sino las coordenadas de las estrellas en la bóveda celeste vistas un punto exterior al grupo de estrellas, para lo cual sí sería más apropiado usar un sistema de coordenadas galáctico con un marco de referencia heliocéntrico.  

Intentaremos generar una maqueta de las estrellas cercanas en la vecindad del sol, a partir de un dataset obtenido de un catálogo real. La muestra de estrellas cercanas en la vecindad de 10 pc, publicada por Reylé et al. [*The 10 parsec sample in the* Gaia *era*](<https://www.aanda.org/articles/aa/pdf/2021/06/aa40985-21.pdf>) (descargable también [aquí](<https://gruze.org/10pc/resources/images/The10pcSample.pdf>)) es el mejor candidato, porque está muy actualizado (23 de abril de 2021), es sencillo, pequeño y contiene todo los parámetros necesarios.
Para ello, es posible utilizar una biblioteca especializada en astronomía como `astroPy`, y comenzaremos importando un módulo para leer el catálogo en la estructura de datos adecuada de Python:

In [ ]:
from astropy.io import ascii

Llamaremos a esta estructura ```nearbyStars```, y cargaremos los datos que, previamente, hemos [descargado](https://gruze.org/10pc) localmente, aquí [./catalog/The10pcSample.csv](./catalog/The10pcSample.csv)

In [ ]:
nearbyStars= ascii.read('./catalog/The10pcSample.csv')

probamos lo que hemos obtenido, con algunas columnas de la tabla que nos interesan. La explicación de las columnas está en el mismo artículo o en este archivo [./catalog/The10pcSample.ReadMe.txt](./catalog/The10pcSample.ReadMe.txt):

In [ ]:
nearbyStars

NB_OBJ,NB_SYS,SYSTEM_NAME,OBJ_CAT,OBJ_NAME,RA,DEC,EPOCH,PARALLAX,PARALLAX_ERROR,PARALLAX_BIBCODE,PMRA,PMRA_ERROR,PMDEC,PMDEC_ERROR,PM_BIBCODE,RV,RV_ERROR,RV_BIBCODE,SP_TYPE,SP_BIBCODE,SP_METHOD,G_CODE,G,G_ESTIMATE,GBP,GRP,U,B,V,R,I,J,H,K,SYSTEM_BIBCODE,EXOPLANET_COUNT,GAIA_DR2,GAIA_EDR3,SIMBAD_NAME,COMMON_NAME,GJ,HD,HIP,COMMENT
int32,int32,str26,str6,str28,float64,float64,float64,float64,float64,str31,float64,float64,float64,float64,str30,float64,float64,str19,str8,str25,str12,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str19,int32,str28,str29,str26,str17,str10,str12,str10,str348
1,1,alf Cen,LM,Proxima Cen,217.392321472009,-62.6760751167667,2016.0,768.066539187357,0.049872905,2020yCat.1350....0G,-3781.74100826516,0.031386077,769.465014647862,0.050524533,2020yCat.1350....0G,-22.345,0.006,2014MNRAS.439.3094B,M5.5,1995AJ....110.1838R,Opt Spec,3,8.984749,--,11.373116,7.5685353,14.21,12.95,11.13,9.45,7.41,5.357,4.835,4.384,2018A&A...615A.172M,1,Gaia DR2 5853498713160606720,Gaia EDR3 5853498713190525696,alf Cen C,Proxima Cen,GJ 551,--,HIP 70890,Proxima Cen c: candidate planet 2019ESS.....410203D
2,1,alf Cen,Planet,Proxima Cen b,217.392321472009,-62.6760751167667,2016.0,768.066539187357,0.049872905,FROM:ProximaCenC,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,2019MNRAS.487..268J,--,--,--,--,--,--,--,--,...
3,1,alf Cen,*,alf Cen A,219.902058331708,-60.83399268831,2000.0,743.0,1.3,2016A&A...586A..90P,-3679.25,3.89,473.67,3.24,2007A&A...474..653V,-22.39,0.004,2016A&A...586A..90P,G2,2006A&A...460..695T,Opt Spec,10,--,-0.09,--,--,0.96,0.72,0.01,--,--,-1.15,-1.38,-1.49,2018A&A...615A.172M,--,--,--,alf Cen A,Rigil Kentaurus,GJ 559 A,HD 128620,HIP 71683,...
4,1,alf Cen,*,alf Cen B,219.896096289873,-60.8375275655841,2000.0,743.0,1.3,2016A&A...586A..90P,-3614.39,20.48,802.98,19.52,2007A&A...474..653V,-22.39,0.004,2016A&A...586A..90P,K1,2006A&A...460..695T,Opt Spec,10,--,1.34,--,--,2.89,2.21,1.33,--,--,-0.01,-0.49,-0.6,2018A&A...615A.172M,--,--,--,alf Cen B,Toliman,GJ 559 B,HD 128621,HIP 71681,...
5,2,Barnard's Star,LM,Barnard's Star,269.448502525438,4.73942005111241,2016.0,546.975939730948,0.040116355,2020yCat.1350....0G,-801.550978368471,0.031820867,10362.3942065466,0.036070455,2020yCat.1350....0G,-110.353,0.0032,2018A&A...616A...7S,M3.5,2015A&A...577A.128A,Opt Spec,3,8.1939745,--,9.791788,6.9580913,12.497,11.24,9.511,8.298,6.741,5.244,4.83,4.524,--,--,Gaia DR2 4472832130942575872,Gaia EDR3 4472832130942575872,Barnard's Star,Barnard's Star,GJ 699,--,HIP 87937,Barnard's Star b: candidate planet 2018Natur.563..365R
6,3,Luhman 16,BD,Luhman 16 A,162.308643668751,-53.3180447534979,2016.0,501.557,0.082,2018A&A...618A.111L,-2759.502,0.147,356.856,0.15,2018A&A...618A.111L,--,--,--,L7.5,2013ApJ...772..129B,NIR Spec,3,16.945,--,20.619625,14.517294,--,16.86,16.2,--,14.95,10.733,9.563,8.841,2013ApJ...767L...1L,--,Gaia DR2 5353626573555863424,Gaia EDR3 5353626573555863424,Luhman 16A,Luhman 16,--,--,--,...
7,3,Luhman 16,BD,Luhman 16 B,162.308402229118,-53.3182778100269,2015.5,501.557,0.082,FROM:Luhman16A,-2759.502,0.147,356.856,0.15,FROM:Luhman16A,--,--,--,T0.5,2013ApJ...772..129B,NIR Spec,2,16.962,--,21.055134,14.453752,--,--,--,--,--,11.22,10.39,9.73,2013ApJ...767L...1L,--,Gaia DR2 5353626573562355584,--,Luhman 16B,Luhman 16,--,--,--,...
8,4,WISEA J085510.74-071442.5,BD,WISEA J085510.74-071442.5,133.780984,-7.243932,2016.7,439.0,2.4,2021ApJS..253....7K,-8123.7,1.3,673.2,1.3,2021ApJS..253....7K,--,--,--,>Y4,2021ApJS..253....7K,NIR Phot,20,--,21.91,--,--,--,--,--,--,--,25.0,23.83,--,--,--,--,--,WISEA J085510.74-071442.5,--,--,--,--,...
9,5,Wolf 359,LM,Wolf 359,164.10319030756,7.00272694098486,2016.0,415.179415678021,0.06837086,2020yCat.1350....0G,-3866.33827514368,0.08130645,-2699.21498767917,0.06910815,2020yCat.1350....0G,19.442,0.032,2020A&A...636A..36L,M6,2019AJ....157...63K,Opt Spec,3,11.038391,--,13.770287,9.58545,16.706,15.541,13.507,11.684,9.507,7.085,6.482,6.084,--

In [ ]:
#nearbyStars['NB_OBJ','NB_SYS','SYSTEM_NAME','RA','DEC','PARALLAX'] #Ej. 'OBJ_CAT','OBJ_NAME',... son los nombres de las columnas del catálogo que necesitamos

Podemos probar que es una genuina estructura de tabla en el interior de Python, interrogando su tipo:

In [ ]:
type(nearbyStars)

astropy.table.table.Table

Ya están los datos del catálogo dentro de Python. Ahora viene...
## La astrofísica
**¿Cómo convertir estos valores en parámetros útiles, con las unidades apropiadas y en los sistemas de coordenadas apropiadoss para calcular y obtener los resultados necesarios para cada objeto?**

Nótese que las columnas RA y DEC están expresadas en grados decimales, pero la columna PARALAX está expresada en centenas, ello nos dice que no está dada en **arcsec**, sino en **mas** (miliarcosegundos). Se impone la necesidad de convertir unidades...
Para ello, astroPy cuenta con un módulo apropiado, con las unidades necesarias para hacer la conversión:

In [ ]:
import astropy.units as u

Sabemos que

$$d (parsec) = 1 / p (arc sec)$$

pero no hay ni que programar esta división: `astroPy` incluso resuelve el problema de hallar el inverso del ángulo de paralaje para obtener la distancia en la unidad que necesitemos (pc, kpc, UA o lyr), sin tener siquiera que dividir, y esto es porque conoce la equivalencia entre las unidades de longitud y los ángulos de paralaje. Una solución cómoda, pero un tanto rara, por lo que vale la pena estudiarse la documentación a ver que otras sorpresas como esta nos aguardan en `astroPy`:

In [ ]:
ProximaCen_parallax= nearbyStars[0]['PARALLAX']

(ProximaCen_parallax * u.mas).to(u.pc, equivalencies=u.parallax())

<Quantity 1.30197053 pc>

note que el valor se multiplica por la unidad en el formato que viene en la tabla y en la conversión `.to()` se expresa en la unidad del resultado esperado como `u.pc`, donde también se agrega la indicación de una equivalencia que astroPy ya conoce, con una función `u.parallax()`.

Y es muy sencillo expresar el resultado en otra unidad:

In [ ]:
(ProximaCen_parallax * u.mas).to(u.lyr, equivalencies=u.parallax())

<Quantity 4.24645992 lyr>

Para obtener las coordendas galácticas, utilizamos una nueva biblioteca:

In [ ]:
#import astropy.coordinates as coord
from astropy.coordinates import SkyCoord

In [ ]:
ProximaCen_coord = SkyCoord(
    ra=nearbyStars[0]['RA']*u.degree,
    dec=nearbyStars[0]['DEC']*u.degree,
    distance=(nearbyStars[0]['PARALLAX'] * u.mas).to(u.pc, equivalencies=u.parallax()),
    frame= 'icrs'
    )

In [ ]:
ProximaCen_coord.ra

<Longitude 217.39232147 deg>

In [ ]:
ProximaCen_coord.galactic

<SkyCoord (Galactic): (l, b, distance) in (deg, deg, pc)
    (313.92550175, -1.91775715, 1.30197053)>

Preparamos un factor de escala para la maqueta, sustituyendo los parsec por una medida apropiada en cm

In [ ]:
scale_factor= 100 * u.cm/u.pc #100 cm:1 pc

In [ ]:
ProximaCen_coord.galactic.cartesian * scale_factor

<CartesianRepresentation (x, y, z) in cm
    (90.27003193, -93.72091668, -4.3570349)>

Ahora vamos a aplicar esta lógica a la tabla completa, para todos aquellos objetos estelares

In [ ]:
from astropy.table import QTable

In [ ]:
nearbyObjects10pc= QTable.read('.\catalog\The10pcSample.csv')

In [ ]:
print ('El catálogo tiene', len(nearbyObjects10pc), 'objetos')

El catálogo tiene 559 objetos


In [ ]:
#star= QTable([obj, sys, sys_name, obj_name, coord], dtype= (int16, int8, u26, u28))
star= QTable()
stars= QTable()

In [ ]:
scale_factor= 10 * u.cm/u.pc #10 cm:1 pc

In [ ]:
for catalogObject in nearbyObjects10pc:
    if not catalogObject['OBJ_CAT'] == 'Planet':
        #print(catalogObject['NB_OBJ', 'NB_SYS', 'OBJ_NAME'])
        star['obj']= [catalogObject['NB_OBJ']]
        star['system']= [catalogObject['NB_SYS']]
        star['system_name']= [catalogObject['SYSTEM_NAME']]
        star['obj_name']= [catalogObject['OBJ_NAME']]
        coord = SkyCoord(
            ra=[catalogObject['RA']]*u.degree,
            dec=[catalogObject['DEC']]*u.degree,
            distance=([catalogObject['PARALLAX']] * u.mas).to(u.pc, equivalencies=u.parallax()),
            frame= 'icrs'
            ).galactic.cartesian * scale_factor
        star['coord']= coord #.to_table
        stars.add_row(star)

ValueError: Mismatch between number of vals and columns

In [ ]:
stars= QTable

In [ ]:
for catalogObject in nearbyObjects10pc:
    if not catalogObject['OBJ_CAT'] == 'Planet':
        #print(catalogObject['NB_OBJ', 'NB_SYS', 'OBJ_NAME'])

        star_coord = SkyCoord(
            ra=[catalogObject['RA']]*u.degree,
            dec=[catalogObject['DEC']]*u.degree,
            distance=([catalogObject['PARALLAX']] * u.mas).to(u.pc, equivalencies=u.parallax()),
            frame= 'icrs'
            ).galactic.cartesian * scale_factor


        star = dict(obj= [catalogObject['NB_OBJ']],
            system= [catalogObject['NB_SYS']],
            system_name= [catalogObject['SYSTEM_NAME']],
            obj_name= [catalogObject['OBJ_NAME']],
            coord= star_coord
            )

        #stars.add_row(star)
        print(star)

{'obj': [1], 'system': [1], 'system_name': ['alf Cen'], 'obj_name': ['Proxima Cen'], 'coord': <CartesianRepresentation (x, y, z) in cm
    [(9.02700319, -9.37209167, -0.43570349)]>}
{'obj': [3], 'system': [1], 'system_name': ['alf Cen'], 'obj_name': ['alf Cen A'], 'coord': <CartesianRepresentation (x, y, z) in cm
    [(9.6373992, -9.39352911, -0.15964954)]>}
{'obj': [4], 'system': [1], 'system_name': ['alf Cen'], 'obj_name': ['alf Cen B'], 'coord': <CartesianRepresentation (x, y, z) in cm
    [(9.63672493, -9.39421262, -0.16013271)]>}
{'obj': [5], 'system': [2], 'system_name': ["Barnard's Star"], 'obj_name': ["Barnard's Star"], 'coord': <CartesianRepresentation (x, y, z) in cm
    [(15.19189606, 9.14605711, 4.44970589)]>}
{'obj': [6], 'system': [3], 'system_name': ['Luhman 16'], 'obj_name': ['Luhman 16 A'], 'coord': <CartesianRepresentation (x, y, z) in cm
    [(5.19760641, -19.16059613, 1.83761581)]>}
{'obj': [7], 'system': [3], 'system_name': ['Luhman 16'], 'obj_name': ['Luhman 16 B'

error uploading: HTTPSConnectionPool(host='api.segment.io', port=443): Max retries exceeded with url: /v1/batch (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000205354DB790>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
